In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import StandardScaler  
from sklearn.model_selection import GridSearchCV


data = pd.read_csv('..\\Fermi-LAT Data\\fl_wrappered.csv',index_col=0)

'''
In 'data' all unknown BCUs are assigned 0. 
All BLLs are assigned 1 and all FSRQs are assigned 2.
Radio Galaxies = 3 and Other = 4
'''

mask1 = data['CLASS1'] == 4
mask2 = data['CLASS1'] == 0
mask3 = data['CLASS1'] == 3
data = data[~mask1]
data = data[~mask2]
data = data[~mask3]

''' JOINING BLLs AND FSRQs INTO A SINGLE CATEGORY '''

#data = data.replace(2,1)

data = data.dropna()

print(len(data['CLASS1']))
data_test = data[~(data['CLASS1'] == 1)]
print(len(data_test['CLASS1']))
print(len(data.columns))

1383
470
11


We will split the dataset (obtained from sklearn load_digits) into a training and test set using the code below 

In [2]:

X = data.loc[:, data.columns != 'CLASS1']
Y = data['CLASS1']

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.30,random_state=4) 
# test_size set the proportion of data to use as test data. The rest of the data will be used as training data

'''HIGHLY RECOMMENDED TO SCALE TRAINING DATA '''

scaler = StandardScaler()  
scaler.fit(x_train)  
x_train = scaler.transform(x_train)  
x_test = scaler.transform(x_test)  

hl_list = []
for i in range(12,14):
    hl_list.append((int(i),int(2)))





In [3]:
from sklearn.model_selection import GridSearchCV

model_parameters = {
        'hidden_layer_sizes':hl_list,
        'activation':['tanh','relu'],
        'solver':['sgd','adam'],
        'alpha':[0.0001,0.001,0.00001],
        'learning_rate':['constant','invscaling','adaptive']
}

clf = GridSearchCV(NN, model_parameters, n_jobs=-1, cv=3)
clf.fit(x_train, y_train)

# Best paramete set
#print('Best parameters found:\n', clf.best_params_)
'''
# All results
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
'''    
#NN.fit(x_train, y_train)

print('Best parameters found:\n', clf.best_params_)

NameError: name 'NN' is not defined

In [4]:
from sklearn.ensemble import BaggingClassifier

NN = MLPClassifier(activation='relu', alpha= 0.0001, hidden_layer_sizes=(13, 2), learning_rate='constant', solver='sgd', random_state=1, max_iter=1000)
NN.fit(x_train, y_train)

'Making use of the above algorithm with bagging implemented'

BNN = BaggingClassifier(estimator=NN,n_estimators=100,bootstrap=True)
BNN.fit(x_train,y_train)

BaggingClassifier(estimator=MLPClassifier(hidden_layer_sizes=(13, 2),
                                          max_iter=1000, random_state=1,
                                          solver='sgd'),
                  n_estimators=100)

Now testing the model:

In [5]:
y_pred = NN.predict(x_test)
y_proba = NN.predict_proba(x_test)

accuracy = accuracy_score(y_test, y_pred)*100
confusion = confusion_matrix(y_test, y_pred)
print('The Neural Network accuracy is ' + str(accuracy))
print('The Neural Network Confusion Matrix is:')
print(confusion)

'''The neural network accuracy initially decreases when bagging
is implemented, suggesting that the data has a high bias'''

b_y_pred = BNN.predict(x_test)
b_y_proba = BNN.predict_proba(x_test)

b_accuracy = accuracy_score(y_test, b_y_pred)*100
b_confusion = confusion_matrix(y_test, b_y_pred)
print('The Bagged Neural Network accuracy is ' + str(b_accuracy))
print('The Bagged Neural Network Confusion Matrix is:')
print(b_confusion)

The Neural Network accuracy is 90.12048192771084
The Neural Network Confusion Matrix is:
[[251  16]
 [ 25 123]]
The Bagged Neural Network accuracy is 90.60240963855422
The Bagged Neural Network Confusion Matrix is:
[[252  15]
 [ 24 124]]


In [ ]:
import matplotlib.pyplot as plt
#print(y_proba)
probs = pd.DataFrame(y_proba,columns=('p_1','p_2'))
bin_probs_1 = pd.cut(probs['p_1'],np.linspace(0,1,25),include_lowest=True)
bin_probs_2 = pd.cut(probs['p_2'],np.linspace(0,1,25),include_lowest=True)

print(x_test.shape)

fig, ax = plt.subplots()
x_axis = np.linspace(0,1,len(bin_probs_1.value_counts()))

ax.bar(x_axis-0.01,bin_probs_1.value_counts(sort=False), width=0.02,label='BLLs')
ax.bar(x_axis+0.01,bin_probs_2.value_counts(sort=False),width=0.02,label='FSRQs')
ax.legend()
ax.set_xlabel('Probability')
ax.set_ylabel('Number of Objects')
ax.set_title('Results of the MLPClassifier algorithm applied to known BLLs and FSRQs')

plt.show()

NameError: name 'pd' is not defined